In [2]:
#Code Taken from:
#ChatGPT
# https://www.geeksforgeeks.org/inverted-index/
# Referenced NLTK documentation 
# HW1


mport nltk
from nltk.corpus import state_union
from nltk.tokenize import word_tokenize
from collections import defaultdict

nltk.download('state_union')
nltk.download('punkt')

def load_state_union_documents():
    documents = state_union.sents()
    return documents

def create_inverted_index(documents):
    inverted_index = defaultdict(list)

    for doc_id, document in enumerate(documents):
        tokens = word_tokenize(' '.join(document))
        for token in set(tokens):  # Use set to remove duplicates within the document
            inverted_index[token].append(doc_id)

    # Sort the postings in increasing order
    for term, postings in inverted_index.items():
        inverted_index[term] = sorted(postings)

    return inverted_index

if __name__ == "__main__":
    # Load documents
    documents = load_state_union_documents()

    # Create inverted index
    inverted_index = create_inverted_index(documents)

    # Print the inverted index
    # for term, postings in inverted_index.items():
    #     print(f"{term}: {postings}")


[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\Aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
# Work Taken from:
# https://nlp.stanford.edu/IR-book/html/htmledition/k-gram-indexes-for-wildcard-queries-1.html
# ChatGPT
# https://www.geeksforgeeks.org/spelling-correction-using-k-gram-overlap/
# https://www.geeksforgeeks.org/python-wildcard-substring-search/



from nltk import ngrams
from nltk.tokenize import word_tokenize
from collections import defaultdict

def create_kgram_index(inverted_index, k=2):
    kgram_index = defaultdict(list)

    for term in inverted_index.keys():
        term = f"#{term}#"  # Add padding to handle edge cases
        kgrams = list(ngrams(term, k))


        
        for kgram in kgrams:
            kgram_str = ''.join(kgram)
            kgram_index[kgram_str].append(term[1:-1])  # Remove padding before storing in index

    return kgram_index

def wildcard_search(query, inverted_index, kgram_index, k=2):
    # Convert the query to lowercase and add padding
    query_padded = f"#{query.lower()}#"
    
    # Generate k-grams for the padded query
    query_kgrams = list(ngrams(query_padded, k))

    # Set to store candidate terms
    candidate_terms = set()

    # Collect terms from k-gram index for each k-gram in the query
    for kgram in query_kgrams:
        kgram_str = ''.join(kgram)
        # Convert k-gram to lowercase before looking it up in the index
        candidate_terms.update(kgram_index.get(kgram_str.lower(), []))

    # Filter terms that match the query (case-insensitive)
    matching_terms = [term[1:-1] for term in candidate_terms if query_padded[1:-1] in term.lower()]

    # Retrieve document identifiers from the inverted index
    document_ids = []
    for term in matching_terms:
        document_ids.extend(inverted_index[term])

    return sorted(list(set(document_ids)))  # Remove duplicates and sort


if __name__ == "__main__":
    # Assuming you already have the inverted index from the previous code
    kgram_index = create_kgram_index(inverted_index, k=2)



    # Perform a single term wildcard search
    query_term = "presi"  # Replace * with any wildcard pattern
    result = wildcard_search(query_term, inverted_index, kgram_index, k=2)

    print(f"Documents matching the wildcard query '{query_term}': {result}")


Documents matching the wildcard query 'presi': [900, 13640]


In [45]:
# Work Taken From
# https://medium.com/@m.derakhshan/some-techniques-for-spelling-correction-bb200a03bd96
# ChatGPT
# https://www.geeksforgeeks.org/spelling-correction-using-k-gram-overlap/


from nltk.metrics import edit_distance

def find_spelling_correction(query, terms):
    # Find the term in 'terms' with the minimum edit distance to the query
    min_distance = float('inf')
    suggestion = None
    
    for term in terms:
        distance = edit_distance(query, term)
        if distance < min_distance:
            min_distance = distance
            suggestion = term
    
    return suggestion

def modified_search(query, inverted_index, kgram_index, terms):
    # Check if the query term is in the inverted index
    if query in inverted_index:
        # Case a: Return documents from the inverted index
        return sorted(inverted_index[query])
    else:
        # Case b: Query term is not in the index, find a spelling correction suggestion
        suggestion = find_spelling_correction(query, terms)
        
        if suggestion is not None:
            return f"Term not found. Did you mean: '{suggestion}'?"
        else:
            return f"Term not found. No spelling suggestions available."

# Example usage:
terms = inverted_index.keys()
query = "HARRY"  # Replace with your query term
result = modified_search(query, inverted_index, kgram_index, terms)

print(f"Search Results for '{query}': {result}")


query = "harry"  # Replace with your query term
result = modified_search(query, inverted_index, kgram_index, terms)

print(f"Search Results for '{query}': {result}")


Search Results for 'HARRY': [0, 116, 1344, 1639, 1915, 2102, 2337]
Search Results for 'harry': Term not found. Did you mean: 'carry'?


In [56]:
# Work taken from:
# ChatGPT
# https://nlp.stanford.edu/IR-book/html/htmledition/blocked-sort-based-indexing-1.html
# https://www.geeksforgeeks.org/understanding-python-pickling-example/

from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize
import pickle
import os
from collections import defaultdict
from ipynb.fs.full.HW1 import invertedIndex

def process_batch_and_write_to_disk(batch_documents, batch_id):
    inverted_index_batch = defaultdict(list)
    
    for doc_id, document in enumerate(batch_documents):
        tokens = word_tokenize(document)
        
        for token in set(tokens):
            inverted_index_batch[token].append(doc_id)
    
    # Write the inverted index batch to disk using pickle
    with open(f'inverted_index_batch_{batch_id}.pkl', 'wb') as f:
        pickle.dump(inverted_index_batch, f)

def merge_pickled_indexes(file_paths):
    merged_index = defaultdict(list)
    
    for file_path in file_paths:
        with open(file_path, 'rb') as f:
            inverted_index_batch = pickle.load(f)
        
        # Merge the inverted index batches
        for term, postings in inverted_index_batch.items():
            merged_index[term].extend(postings)
    
    # Sort the postings for each term
    merged_index = {term: sorted(postings) for term, postings in merged_index.items()}
    
    return merged_index

# Load the 20newsgroups dataset
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

# Define the batch size
batch_size = 500

# Process the data in batches and write inverted index to disk
for batch_id in range(0, len(documents), batch_size):
    batch_documents = documents[batch_id : batch_id + batch_size]
    process_batch_and_write_to_disk(batch_documents, batch_id)

# List all pickle files in the current directory
pickle_files = [file for file in os.listdir() if file.endswith('.pkl')]

# Merge the pickled indexes
merged_index = merge_pickled_indexes(pickle_files)

# Compare the previous index with the BSB merged index
# ... (your comparison logic here)


print("Inverted Index for 'chicago':", invertedIndex.get('chicago', []))


print("Merged Index for Chicago:", merged_index.get('chicago'))

print("")

print("Size of HW1 Inverted Index:", len(invertedIndex))
print("Size of HW2 Merge Index:", len(merged_index))




Inverted Index for 'chicago': [1844, 6967, 7269, 9076]
Merged Index for Chicago: [70, 351, 362]

Size of HW1 Inverted Index: 215552
Size of HW2 Merge Index: 161732
